In [13]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [9]:
state = pd.read_csv("/home/harshit/geography/district.csv")
state = list(set(state["STATE"]))
state.sort()
stateer = pd.read_csv("/home/harshit/geography/ercompanies.csv")
stateer=stateer.replace("\+ACY-","&",regex=True)
hq_path = "/home/harshit/geography/hqportfolios/"

for h in state:
    date = pd.DataFrame(columns=['Month'])
    month = pd.DataFrame(columns=['Month'])
    if(os.path.exists(hq_path+h+".csv")==True):
        hqread = pd.read_csv(hq_path+h+".csv")
        date = pd.DataFrame(columns=['Month'])
        for j in hqread["Company"]:
            j=j.upper()
            df = pd.read_csv("/home/harshit/geography/mret/"+j+".csv")
            df.rename(columns={"Return":j+"HQ"},inplace=True)
            date = pd.merge(date,df,left_on="Month", right_on="Month",how="outer",sort=False)
        date["meanHQ"] = round(np.mean(date,axis=1),2)
        date = date[:-3]
    
    if(h in stateer):
        comp=stateer[h][pd.notna(stateer[h])]
        month = pd.DataFrame(date["Month"])
        for i in comp:
            i=i.upper()
            ret = pd.read_csv("/home/harshit/geography/mret/"+i[0:-2]+".csv")
            ret.rename(columns={"Return":i+"ER"},inplace=True)

            if "14" in i:
                if("February-2014" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="February-2014"].index.values[0]:ret.loc[ret["Month"]=="December-2014"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2014"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")


            if "15" in i:
                if("January-2015" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2015"].index.values[0]:ret.loc[ret["Month"]=="December-2015"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2015"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")

            if "16" in i:
                if("January-2016" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2016"].index.values[0]:ret.loc[ret["Month"]=="December-2016"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2016"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")


            if "17" in i:
                if("January-2017" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2017"].index.values[0]:ret.loc[ret["Month"]=="December-2017"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2017"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")


            if "18" in i:
                if("January-2018" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2018"].index.values[0]:ret.loc[ret["Month"]=="December-2018"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2018"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")        

        
        month["meanER"] = round(np.mean(month,axis=1),2)
        
        
    date = pd.merge(date,month,left_on="Month",right_on="Month",how="outer",sort=False)  
    date = date[:-3]
    if(len(date!=0)):
        date.to_csv("/home/harshit/geography/meanret/"+h+".csv",index=False)

In [11]:
date=date["Month"]

In [20]:
eth =  pd.to_datetime(date, format='%B-%Y')

In [27]:
type(eth)

pandas.core.series.Series